In [1]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요"

In [2]:
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [3]:
# 단어사전 구축 및 단어별 인덱스 부여해주기
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

{'오늘': 0, '날씨': 1, '구름': 2}


In [4]:
# 원핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [9]:
# 원핫인코딩은 단점이 많음... => 분산표현 방식을 채택하는 것으로~~

In [17]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# 데이터 읽어오기
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] #헤더제거
    return data
# 학습시간 측정
start = time.time()

# 리뷰파일 읽기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data))
print('1) 말뭉치 데이터 읽기 완료:', time.time() - start)

# 문장 단위로 명사만 추출해 학습 입력 데이터로 만들기
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료:', time.time() - start)

# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)
print('3) Word2Vec 모델 학습 완료:', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료:', time.time() - start)

# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print('corpus count:', model.corpus_count) # 말뭉치 수
print('corpus total words:', model.corpus_total_words) # 단어 수

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료: 2.1409778594970703
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료: 72.15831160545349
3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 완료: 85.46841812133789
4) 학습된 모델 저장
4) 학습도니 모델 저장 완료: 85.7264769077301
corpus count: 200000
corpus total words: 1076899


In [19]:
model = Word2Vec.load('nvmc.model')
print('Corpus_total_words:', model.corpus_total_words)

Corpus_total_words: 1076899


In [20]:
# '사랑'이라는 단어로 생성한 단어 임베딩 벡터
print('사랑:', model.wv['사랑'])

사랑: [-0.09963502 -0.51247454  0.30768812 -0.07181589  0.16950014 -0.27226546
 -0.20034589  0.20808795 -0.0278259   0.31236452  0.05347251 -0.05940739
  0.29611224 -0.0622078  -0.16552979  0.32803574 -0.09007484 -0.07510316
  0.14407587 -0.31191877  0.25298133  0.16413441 -0.3008848   0.20796376
 -0.40281355 -0.19278833  0.00496402 -0.1483992  -0.17066455 -0.21005322
 -0.00663083 -0.10731995 -0.03389509 -0.11334454  0.0316027   0.25160068
  0.12745164  0.02471401  0.02739664  0.00912736 -0.2070161  -0.1329094
 -0.43163353 -0.48274696  0.5291333   0.13072701  0.03685965 -0.00268602
  0.3482423   0.00807548  0.08908078 -0.31108728 -0.02584817  0.03388226
  0.15197004 -0.07177763  0.23828048 -0.26313645  0.30797178  0.03409517
 -0.06699759  0.1264337  -0.41427356  0.14593731 -0.13872607 -0.18302204
 -0.33259308  0.0593227   0.02350021  0.24098799 -0.2289866  -0.39240658
  0.03107566  0.11131877  0.13886136  0.0052525   0.25983262 -0.2097386
 -0.20219477  0.01825125  0.12977855 -0.05343376 

In [21]:
# 단어 유사도 계산해보기
print('일요일 = 월요일\t', model.wv.similarity(w1='일요일', w2='월요일'))
print('안성기 = 배우\t', model.wv.similarity(w1='안성기', w2='배우'))
print('대기업 = 삼성\t', model.wv.similarity(w1='대기업', w2='삼성'))
print('일요일 != 삼성\t', model.wv.similarity(w1='일요일', w2='삼성'))
print('히어로 != 삼성\t', model.wv.similarity(w1='히어로', w2='삼성'))

일요일 = 월요일	 0.6136624
안성기 = 배우	 0.51435363
대기업 = 삼성	 0.53871554
일요일 != 삼성	 0.27813858
히어로 != 삼성	 0.1362387


In [22]:
# 가장 유사한 단어 추출
print(model.wv.most_similar('안성기', topn=5))
print(model.wv.most_similar('시리즈', topn=5))

[('장미희', 0.6895906329154968), ('현영', 0.6719661951065063), ('조한선', 0.6692084670066833), ('박신양', 0.6639488339424133), ('김희원', 0.6611346006393433)]
[('더 울버린', 0.6350187063217163), ('캐리비안의 해적', 0.6309864521026611), ('나니아 연대기', 0.6261622905731201), ('엑스맨', 0.6226657032966614), ('러시아워', 0.6181084513664246)]
